### 1. Import

In [1]:
import os
import json

from langchain_openai import ChatOpenAI
from ragas.llms import LangchainLLMWrapper

from document_processor import TextProcessor, ImageProcessor, PageImageProcessor, ImageTextualSummaryProcessor
from multimodal_rag import MultimodalRAG
from embedder import OpenAIEmbedder, ColPaliEmbedder
from pdf_to_qa import generate_qa_for_pdf
from evaluation import evaluate_generation, compute_mrr_at_k, compute_recall_at_k
from datasets import load_dataset
from chartQAloader import generate_chartQA_pdf_and_json

### 2. Configuration

In [ ]:
#PDF_FILE = "knowledge/subset_riksbanken.pdf"
#PDF_FILE = "knowledge/ChartQA_Evaluation_Set.pdf"
PDF_FILE = "knowledge/subset_ChartQA_Evaluation_Set.pdf"

#text_processor = TextProcessor(OpenAIEmbedder())
#image_processor = ImageProcessor(ColPaliEmbedder())
page_image_processor = PageImageProcessor(ColPaliEmbedder())
#image_textual_summary_processor = ImageTextualSummaryProcessor(OpenAIEmbedder(), no=1) #Select no 3 for chartQA emb. 1 or 2 for other emb.

rag = MultimodalRAG([page_image_processor], PDF_FILE)
evaluator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4o-mini")) # For Ragas evaluation

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

### 3. Generate Dataset

#### ChartQA

In [ ]:
# Load ChartQA from huggingface
dataset = load_dataset('lmms-lab/ChartQA', split='test')
subset_dataset = dataset.select(range(10))

# Generate PDF of ChartQA and JSON dataset
generate_chartQA_pdf_and_json(subset_dataset, pdf_output_path='knowledge/subset_ChartQA_Evaluation_Set.pdf', json_output_path='QA_subset_ChartQA_Evaluation_Set.json')

Processing Charts: 100%|██████████| 10/10 [00:01<00:00,  8.42chart/s]

PDF saved as knowledge/subset_ChartQA_Evaluation_Set.pdf
JSON file saved as QA_subset_ChartQA_Evaluation_Set.json


#### CoinQA

In [ ]:
# Check if QA file already exists
qa_filepath = "QA_" + os.path.basename(PDF_FILE).replace('.pdf', '.json')

if os.path.exists(qa_filepath):
    qa_path = qa_filepath
    print(f"Using existing QA file: {qa_path}")
else:
    qa_path = generate_qa_for_pdf(PDF_FILE)
    print(f"Generated new QA file: {qa_path}")

Using existing QA file: QA_subset_ChartQA_Evaluation_Set.json


### 4. Answering the QA

In [6]:
with open(qa_path, 'r', encoding='utf-8') as f:
    qa_data = json.load(f)

# Generate dataset
rag_generated_answers = []

# Check if generated answers file already exists
rag_answers_path = "rag_generated_answers_" + os.path.basename(qa_path)

if os.path.exists(rag_answers_path):
    rag_generated_answers = json.load(open(rag_answers_path, 'r', encoding='utf-8'))
    print(f"Using existing RAG generated answers file: {rag_answers_path}")
else:
    for qa in qa_data:
        query = qa["question"]
        reference = qa["answer"]

        relevant_docs = rag.get_most_relevant_docs(query, top_k=3)
        response = rag.generate_answer(query, relevant_docs)
        rag_generated_answers.append(
            {
                "query":query,
                "retrieved_contexts":relevant_docs,
                "generated_answer":response,
                "true_answer":reference
            }
        )

    # Save the dataset to a JSON file
    output_dataset_file = "rag_generated_answers_" + os.path.basename(qa_path)

    with open(output_dataset_file, 'w', encoding='utf-8') as f:
        json.dump(rag_generated_answers, f, ensure_ascii=False, indent=4)
    print(f"Generated new RAG generated answers file: {output_dataset_file}")

Generated new RAG generated answers file: rag_generated_answers_QA_subset_ChartQA_Evaluation_Set.json


### 5. Evaluate Retrieval

In [ ]:
all_real_pages, all_retrieved_pages = [], []

for rag_answer in rag_generated_answers:
    real_page = next(
        item["page_number"] for item in qa_data if item["question"] == rag_answer["query"]
    )
    retrieved_pages = [doc["page_number"] for doc in rag_answer["retrieved_contexts"]]
    
    all_real_pages.append(real_page)
    all_retrieved_pages.append(retrieved_pages)

for k in [3, 5]:
    mrr = compute_mrr_at_k(all_retrieved_pages, all_real_pages, k)
    recall = compute_recall_at_k(all_retrieved_pages, all_real_pages, k)
    print(f"MRR@{k}: {mrr}")
    print(f"Recall@{k}: {recall}")


### 6. Evaluate Generation

In [ ]:
faithfulness_and_relevance = evaluate_generation(rag_generated_answers, evaluator_llm)
print(faithfulness_and_relevance)